# ✅ Imports

In [1]:
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import load_model
import numpy as np
import os


2025-04-15 14:21:18.349580: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744726878.612004      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744726878.689520      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
os.listdir('/kaggle/input') 

['cifake-real-and-ai-generated-synthetic-images',
 'vgg19_adv_cifake_epsilon0.05']

# ✅ Model Builder

In [5]:
def build_vgg19():
    base = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base.layers:
        layer.trainable = False
    x = Flatten()(base.output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(1, activation='sigmoid')(x)
    return Model(base.input, out)

# ✅ Data Generators

In [6]:
import shutil
from sklearn.model_selection import train_test_split

base_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train"
val_dir = "/kaggle/working/validation"

for label in ['REAL', 'FAKE']:
    os.makedirs(os.path.join(val_dir, label), exist_ok=True)
    files = os.listdir(os.path.join(base_dir, label))
    train_files, val_files = train_test_split(files, test_size=0.2, random_state=42)

    for fname in val_files:
        src = os.path.join(base_dir, label, fname)
        dst = os.path.join(val_dir, label, fname)
        shutil.copyfile(src, dst)


In [7]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train',
    target_size=(224, 224), batch_size=32, class_mode='binary', shuffle=True)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    '/kaggle/working/validation',
    target_size=(224, 224), batch_size=32, class_mode='binary', shuffle=False)

test_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test',
    target_size=(224, 224), batch_size=32, class_mode='binary', shuffle=False)

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


# ✅ FGSM Attack Function and PGD Attack Function

In [8]:
@tf.function
def fgsm_attack(model, images, labels, epsilon=0.01):
    with tf.GradientTape() as tape:
        tape.watch(images)
        predictions = model(images)
        loss = tf.keras.losses.BinaryCrossentropy()(labels, predictions)
    gradient = tape.gradient(loss, images)
    signed_grad = tf.sign(gradient)
    adv_images = images + epsilon * signed_grad
    return tf.clip_by_value(adv_images, 0, 1)

# ✅ PGD Attack Function
@tf.function
def pgd_attack(x, y, model, loss_fn, epsilon=0.01, alpha=0.007, iters=10):
    x_adv = tf.identity(x)

    for i in range(iters):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            prediction = model(x_adv, training=False)
            loss = loss_fn(y, prediction)

        grad = tape.gradient(loss, x_adv)
        signed_grad = tf.sign(grad)
        x_adv = x_adv + alpha * signed_grad
        x_adv = tf.clip_by_value(x_adv, x - epsilon, x + epsilon)
        x_adv = tf.clip_by_value(x_adv, 0.0, 1.0)  # ensure valid pixel range

    return x_adv


# ✅ Train and Save Baseline Model

In [11]:
# Paths to save model and weights
weights_path_baseline = '/kaggle/working/vgg19_baseline_cifake.weights.h5'
model_path_baseline   = '/kaggle/working/vgg19_baseline_cifake_model.h5'
#Kaggle
if os.path.exists('/kaggle/input/vgg19_adv_cifake_epsilon0.05/tensorflow2/default/1/vgg19_baseline_cifake_model.h5'):
    print("Loading saved model...")
    baseline_model = load_model('/kaggle/input/vgg19_adv_cifake_epsilon0.05/tensorflow2/default/1/vgg19_baseline_cifake_model.h5')
#Colab
# if os.path.exists(model_path_baseline):
#     print("Loading saved model...")
#     baseline_model = load_model(model_path_baseline)
else:
    print("Training baseline model...")
    baseline_model = build_vgg19()
    baseline_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    baseline_model.fit(train_gen, epochs=3, validation_data=val_gen)
    baseline_model.save_weights(weights_path_baseline)
    baseline_model.save(model_path_baseline)
    print("Baseline model trained and saved.")

print("\nEvaluating Baseline Model on Clean Test Set:")
baseline_clean_acc = baseline_model.evaluate(test_gen)
print("Baseline Clean Accuracy:", baseline_clean_acc)


Loading saved model...

Evaluating Baseline Model on Clean Test Set:


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


625/625 ━━━━━━━━━━━━━━━━━━━━ 130s 206ms/step - accuracy: 0.8809 - loss: 0.2490
Baseline Clean Accuracy: [0.20891635119915009, 0.913349986076355]


In [9]:
# Paths to save model and weights
weights_path_baseline = '/kaggle/working/vgg19_baseline_cifake.weights.h5'
model_path_baseline   = '/kaggle/working/vgg19_baseline_cifake_model.h5'

#Kaggle
if os.path.exists('/kaggle/input/vgg19_adv_cifake_epsilon0.05/tensorflow2/default/1/vgg19_baseline_cifake_model.h5'):
    print("Loading saved model...")
    baseline_model = load_model('/kaggle/input/vgg19_adv_cifake_epsilon0.05/tensorflow2/default/1/vgg19_baseline_cifake_model.h5')
else:
    print("Training baseline model...")
    baseline_model = build_vgg19()
    baseline_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    baseline_model.fit(train_gen, epochs=3, validation_data=val_gen)
    baseline_model.save_weights(weights_path_baseline)
    baseline_model.save(model_path_baseline)
    print("Baseline model trained and saved.")

# Get a batch of test data
test_images, test_labels = next(test_gen)
test_images = tf.convert_to_tensor(test_images)
test_labels = tf.convert_to_tensor(test_labels)



# Define epsilon values to test
epsilons = [0.01, 0.03, 0.05, 0.1]

# Evaluation function using YOUR original attack functions
def evaluate_attacks(model, images, labels, epsilons):
    for eps in epsilons:
        print(f"\nEvaluating for ε = {eps:.2f}")
        
        # FGSM Evaluation
        adv_images_fgsm = fgsm_attack(model, images, labels, epsilon=eps)
        _, acc_fgsm = model.evaluate(adv_images_fgsm, labels, verbose=0)
        print(f"FGSM Accuracy (ε={eps:.2f}): {acc_fgsm:.4f}")
        
        # PGD Evaluation (using YOUR original PGD function)
        adv_images_pgd = pgd_attack(images, labels, model, 
                                  loss_fn=tf.keras.losses.BinaryCrossentropy(),
                                  epsilon=eps, alpha=0.007, iters=10)
        _, acc_pgd = model.evaluate(adv_images_pgd, labels, verbose=0)
        print(f"PGD Accuracy (ε={eps:.2f}): {acc_pgd:.4f}")

# Run evaluations
print("\nEvaluating Adversarial Robustness:")
evaluate_attacks(baseline_model, test_images, test_labels, epsilons)

Loading saved model...


I0000 00:00:1744729128.509399      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1744729128.510207      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5



Evaluating Baseline Model on Clean Test Set:


I0000 00:00:1744729132.466827     122 service.cc:148] XLA service 0x78cde8106180 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744729132.467833     122 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1744729132.467854     122 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1744729132.715158     122 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.9375 - loss: 0.3287


I0000 00:00:1744729143.111693     122 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Baseline Clean Accuracy: 0.9375

Evaluating Adversarial Robustness:

Evaluating for ε = 0.01
FGSM Accuracy (ε=0.01): 0.1250
PGD Accuracy (ε=0.01): 0.1250

Evaluating for ε = 0.03
FGSM Accuracy (ε=0.03): 0.1250
PGD Accuracy (ε=0.03): 0.0938

Evaluating for ε = 0.05
FGSM Accuracy (ε=0.05): 0.1250
PGD Accuracy (ε=0.05): 0.0938

Evaluating for ε = 0.10
FGSM Accuracy (ε=0.10): 0.1250
PGD Accuracy (ε=0.10): 0.0938


In [13]:
# Get test data (EXACTLY as you specified)
x_test_sample, y_test_sample = next(test_gen)
x_test_tensor = tf.convert_to_tensor(x_test_sample, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test_sample, dtype=tf.float32)

# Evaluate clean accuracy first
print("\nEvaluating Baseline Model on Clean Test Set:")
clean_loss, clean_acc = baseline_model.evaluate(x_test_tensor, y_test_tensor)
print(f"Clean Accuracy: {clean_acc:.4f}")

# Define epsilon values
epsilons = [0.01, 0.03, 0.05, 0.1]

# Evaluate for each epsilon (BUILDING ON YOUR EXACT STRUCTURE)
for eps in epsilons:
    print(f"\n=== Evaluating for ε = {eps:.2f} ===")
    
    # FGSM (YOUR EXACT FORMAT)
    x_fgsm = fgsm_attack(baseline_model, x_test_tensor, y_test_tensor, epsilon=eps)
    print("\nEvaluating Model on FGSM-Adversarial Examples:")
    fgsm_loss, fgsm_acc = baseline_model.evaluate(x_fgsm, y_test_tensor)
    print(f"FGSM Accuracy (ε={eps:.2f}): {fgsm_acc:.4f}")
    
    # PGD (using your original parameters)
    x_pgd = pgd_attack(x_test_tensor, y_test_tensor, baseline_model,
                      loss_fn=tf.keras.losses.BinaryCrossentropy(),
                      epsilon=eps, alpha=0.007, iters=10)
    print("\nEvaluating Model on PGD-Adversarial Examples:")
    pgd_loss, pgd_acc = baseline_model.evaluate(x_pgd, y_test_tensor)
    print(f"PGD Accuracy (ε={eps:.2f}): {pgd_acc:.4f}")


Evaluating Baseline Model on Clean Test Set:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step - accuracy: 0.8750 - loss: 0.2359
Clean Accuracy: 0.8750

=== Evaluating for ε = 0.01 ===

Evaluating Model on FGSM-Adversarial Examples:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step - accuracy: 0.1562 - loss: 1.0998
FGSM Accuracy (ε=0.01): 0.1562

Evaluating Model on PGD-Adversarial Examples:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step - accuracy: 0.1562 - loss: 1.1125
PGD Accuracy (ε=0.01): 0.1562

=== Evaluating for ε = 0.03 ===

Evaluating Model on FGSM-Adversarial Examples:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.1562 - loss: 1.1002
FGSM Accuracy (ε=0.03): 0.1562

Evaluating Model on PGD-Adversarial Examples:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.1250 - loss: 1.1389
PGD Accuracy (ε=0.03): 0.1250

=== Evaluating for ε = 0.05 ===

Evaluating Model on FGSM-Adversarial Examples:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step - accuracy: 0.1562 - loss: 1.1003
FGSM Accuracy (ε=0.05): 0.1562

Ev

# ✅ FGSM Vulnerability Evaluation

In [14]:
x_test_sample, y_test_sample = next(test_gen)
x_test_tensor = tf.convert_to_tensor(x_test_sample, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test_sample, dtype=tf.float32)

x_fgsm_test = fgsm_attack(baseline_model, x_test_tensor, y_test_tensor)
print("\nEvaluating Baseline Model on FGSM-Adversarial Examples:")
baseline_fgsm_acc = baseline_model.evaluate(x_fgsm_test, y_test_sample)
print("Baseline Accuracy on FGSM-Adversarial:", baseline_fgsm_acc)


Evaluating Baseline Model on FGSM-Adversarial Examples:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - accuracy: 0.1250 - loss: 1.1114
Baseline Accuracy on FGSM-Adversarial: [1.1114184856414795, 0.125]


# Generate and Save Perturbed Data (FGSM + PGD)

In [17]:
import tensorflow as tf
import numpy as np
import os
from tqdm import tqdm

# Directories
base_train_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train"
perturbed_dir = "/kaggle/working/train_perturbed_only"  # All perturbed, no clean data
weights_path_baseline = "/kaggle/input/vgg19_adv_cifake_epsilon0.05/tensorflow2/default/1/vgg19_baseline_cifake_model.h5"

# Create directories
os.makedirs(os.path.join(perturbed_dir, 'REAL'), exist_ok=True)
os.makedirs(os.path.join(perturbed_dir, 'FAKE'), exist_ok=True)

# Load baseline model (for generating perturbations)
baseline_model = build_vgg19()
baseline_model.load_weights(weights_path_baseline)  # Pretrained weights

# Data generator (no shuffling for consistent file mapping)
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    base_train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Critical for matching files to paths
)

# Attack parameters
epsilon_fgsm = 0.05
epsilon_pgd = 0.03
alpha_pgd = 0.007
iters_pgd = 10
loss_fn = tf.keras.losses.BinaryCrossentropy()

# Generate and save perturbed data
num_batches = len(train_gen)

for i in tqdm(range(num_batches)):
    x_batch, y_batch = next(train_gen)
    
    # Get indices and paths for this batch
    batch_indices = train_gen.index_array[i*32 : (i+1)*32]
    paths = [train_gen.filepaths[idx] for idx in batch_indices]

    # Split batch: 50% FGSM, 50% PGD
    split = len(x_batch) // 1

    # Generate FGSM samples
    x_fgsm = fgsm_attack(
        baseline_model, 
        tf.convert_to_tensor(x_batch[:split]), 
        tf.convert_to_tensor(y_batch[:split]), 
        epsilon_fgsm
    ).numpy()

    # Generate PGD samples
    # x_pgd = pgd_attack(
    #     tf.convert_to_tensor(x_batch[split:]), 
    #     y_batch[split:], 
    #     baseline_model, 
    #     loss_fn,
    #     epsilon=epsilon_pgd, 
    #     alpha=alpha_pgd, 
    #     iters=iters_pgd
    # ).numpy()

    # Combine and save
    # x_perturbed = np.concatenate([x_fgsm, x_pgd], axis=0)
    x_perturbed = x_fgsm
    # Save images
    for j, path in enumerate(paths):
        class_name = 'REAL' if 'REAL' in path else 'FAKE'
        filename = os.path.basename(path).replace('.jpg', f'_perturbed_{i}_{j}.jpg')
        save_path = os.path.join(perturbed_dir, class_name, filename)
        tf.keras.preprocessing.image.save_img(save_path, x_perturbed[j])

Found 100000 images belonging to 2 classes.


  2%|▏         | 64/3125 [00:36<28:54,  1.76it/s] 


KeyboardInterrupt: 

In [28]:
import tensorflow as tf
import numpy as np
import os
from tqdm import tqdm
import random

# --- Original directory setup ---
base_train_dir = "/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train"
perturbed_dir = "/kaggle/working/train_perturbed_only"
weights_path_baseline = "/kaggle/input/vgg19_adv_cifake_epsilon0.05/tensorflow2/default/1/vgg19_baseline_cifake_model.h5"

# Correct directory creation
os.makedirs(os.path.join(perturbed_dir, 'REAL'), exist_ok=True)
os.makedirs(os.path.join(perturbed_dir, 'FAKE'), exist_ok=True)
# --- Load model --- 
baseline_model = build_vgg19()
baseline_model.load_weights(weights_path_baseline)

# --- Get all file paths ---
all_filepaths = []
for root, dirs, files in os.walk(base_train_dir):
    for file in files:
        if file.endswith('.jpg'):
            all_filepaths.append(os.path.join(root, file))

# Select 1/4 of files (preserving class balance)
real_files = [f for f in all_filepaths if 'REAL' in f]
fake_files = [f for f in all_filepaths if 'FAKE' in f]
selected_files = (
    random.sample(real_files, len(real_files)//4) + 
    random.sample(fake_files, len(fake_files)//4)
)

print(f"Original dataset: {len(all_filepaths)} images")
print(f"Using subset: {len(selected_files)} images (1/4 of original)\n")

# --- Create custom generator for subset ---
class SubsetGenerator(ImageDataGenerator):
    def flow_from_directory(self, directory, subset_files, **kwargs):
        generator = super().flow_from_directory(directory, **kwargs)
        # Filter to only include our selected files
        mask = [f in subset_files for f in generator.filepaths]
        generator.filepaths = [f for f, m in zip(generator.filepaths, mask) if m]
        generator.classes = [c for c, m in zip(generator.classes, mask) if m]
        generator.samples = len(generator.filepaths)
        return generator

# Create generator with subset
train_gen = SubsetGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    base_train_dir,
    subset_files=selected_files,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# --- Processing loop with progress bar ---
num_batches = len(train_gen)
print(f"\nProcessing {num_batches} batches...")
with tqdm(total=num_batches, unit='batch') as pbar:
    for i in range(num_batches):
        x_batch, y_batch = next(train_gen)
        
        # Generate FGSM samples
        x_fgsm = fgsm_attack(
            baseline_model, 
            tf.convert_to_tensor(x_batch), 
            tf.convert_to_tensor(y_batch), 
            epsilon=0.05
        ).numpy()

        # Save images
        batch_indices = train_gen.index_array[i*32 : (i+1)*32]
        paths = [train_gen.filepaths[idx] for idx in batch_indices]
        for j, path in enumerate(paths):
            class_name = 'REAL' if 'REAL' in path else 'FAKE'
            filename = os.path.basename(path).replace('.jpg', f'_perturbed_{i}_{j}.jpg')
            save_path = os.path.join(perturbed_dir, class_name, filename)
            tf.keras.preprocessing.image.save_img(save_path, x_fgsm[j])
        
        pbar.update(1)






KeyboardInterrupt: 

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img

# --- Custom data loading ---
def load_and_preprocess_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = preprocess_input(img)
    return img

# Process images in batches
batch_size = 32
num_batches = len(selected_files) // batch_size

print(f"Processing {num_batches} batches...")
with tqdm(total=num_batches, unit='batch') as pbar:
    for i in range(num_batches):
        batch_files = selected_files[i*batch_size : (i+1)*batch_size]
        
        # Load and preprocess batch
        x_batch = np.array([load_and_preprocess_image(f).numpy() for f in batch_files])
        y_batch = np.array([0 if 'REAL' in f else 1 for f in batch_files])
        
        # Generate FGSM samples
        x_fgsm = fgsm_attack(
            baseline_model,
            tf.convert_to_tensor(x_batch),
            tf.convert_to_tensor(y_batch),
            epsilon=0.05
        ).numpy()
        
        # Save images
        for j, path in enumerate(batch_files):
            class_name = 'REAL' if 'REAL' in path else 'FAKE'
            filename = os.path.basename(path).replace('.jpg', f'_perturbed_{i}_{j}.jpg')
            save_path = os.path.join(perturbed_dir, class_name, filename)
            array_to_img(x_fgsm[j]).save(save_path)
        
        pbar.update(1)

Processing 781 batches...


100%|██████████| 781/781 [09:32<00:00,  1.37batch/s]


# Train on Perturbed Data

In [24]:
perturbed_gen = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
    perturbed_dir,  # Directory with 100% perturbed data
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

# Initialize model (same architecture as baseline)
adv_model = build_vgg19()
adv_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train exclusively on adversarial data
adv_model.fit(perturbed_gen, epochs=3, validation_data=val_gen)

Found 27040 images belonging to 2 classes.
Epoch 1/3
845/845 ━━━━━━━━━━━━━━━━━━━━ 313s 366ms/step - accuracy: 0.7232 - loss: 2.5260 - val_accuracy: 0.5095 - val_loss: 3.6678
Epoch 2/3
845/845 ━━━━━━━━━━━━━━━━━━━━ 308s 365ms/step - accuracy: 0.8128 - loss: 0.4012 - val_accuracy: 0.5042 - val_loss: 5.8864
Epoch 3/3
845/845 ━━━━━━━━━━━━━━━━━━━━ 309s 365ms/step - accuracy: 0.8362 - loss: 0.3519 - val_accuracy: 0.5140 - val_loss: 5.3942


# Saving the Model

In [27]:
weights_path_adv = '/kaggle/working/vgg19_adv_FGSMonly_cifake.weights.h5'
model_path_adv   = '/kaggle/working/vgg19_adv_FGSMonly_cifake_model.h5'

adv_model.save_weights(weights_path_adv)
adv_model.save(model_path_adv)

# Evaluate Robustness

In [29]:
# Evaluate on clean test data (optional)
clean_loss, clean_acc = adv_model.evaluate(test_gen, verbose=0)
print(f"Clean Test Accuracy: {clean_acc:.4f}")

# Evaluate on FGSM-attacked test data
x_test, y_test = next(test_gen)
x_fgsm_test = fgsm_attack(adv_model, tf.convert_to_tensor(x_test), tf.convert_to_tensor(y_test), epsilon_fgsm)
fgsm_loss, fgsm_acc = adv_model.evaluate(x_fgsm_test, y_test, verbose=0)
print(f"FGSM Test Accuracy: {fgsm_acc:.4f} (Baseline was ~12.5%)")

Clean Test Accuracy: 0.5147
FGSM Test Accuracy: 0.2188 (Baseline was ~12.5%)
